In [4]:

import os
import time
import re
import random
import zipfile
import shutil
import math
import json
import logging
from typing import List, Set, Optional, Tuple
from urllib.parse import quote
from io import BytesIO

import ddddocr
from PIL import Image
import undetected_chromedriver as uc  # 抗封鎖核心驅動
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    NoSuchElementException,
    TimeoutException,
    NoAlertPresentException,
    WebDriverException,
    UnexpectedAlertPresentException  # 已補上：防止警告視窗導致崩潰
)

# 獲取腳本所在的目錄
BASE_DIR = os.getcwd()

class BaseThesisDownloader:
    """
    [強健版下載器] 
    整合自動清除 Cookie、智慧冷卻與抗封鎖機制。
    """
    def __init__(self,
                 keyword: str,
                 download_dir: str = "downloaded_theses",
                 log_file: str = "download_log.txt",
                 page_progress_file: str = "page_progress.txt",
                 max_downloads_per_session: int = 70,
                 items_per_page: int = 10,
                 inter_article_sleep_range: Tuple[float, float] = (10.0, 20.0),
                 inter_page_sleep_range: Tuple[float, float] = (20.0, 45.0)
                 ):
        self.base_url = "https://ndltd.ncl.edu.tw/cgi-bin/gs32/gsweb.cgi/login?o=dwebmge"
        self.keyword = keyword
        self.download_dir = os.path.join(BASE_DIR, download_dir)
        self.log_file = os.path.join(BASE_DIR, log_file)
        self.page_progress_file = os.path.join(BASE_DIR, page_progress_file)
        self.max_downloads_per_session = max_downloads_per_session
        self.items_per_page = items_per_page
        self.inter_article_sleep_range = inter_article_sleep_range
        self.inter_page_sleep_range = inter_page_sleep_range
        self.downloaded_urls, self.last_crawled_page = self._load_log()
        self.session_download_count = 0
        self.total_pages = 0
        self.driver = None
        self.wait = None
        self.main_window_handle = None
        
        print("[-] 正在初始化 ddddocr 引擎...")
        self.ocr = ddddocr.DdddOcr(show_ad=False)
        print("[*] ddddocr 引擎初始化完成。")
        print(f"[*] 本次執行最大下載量設定為: {self.max_downloads_per_session} 篇")

    def _normalize_url(self, url: str) -> Optional[str]:
        if not isinstance(url, str): return None
        match = re.search(r'/record\?.*$', url)
        return match.group(0) if match else None

    def _setup_driver(self):
        print("[-] 設定 Selenium WebDriver (Anti-Ban Mode)...")
        os.makedirs(self.download_dir, exist_ok=True)
        
        # 設定 undetected_chromedriver 選項
        options = uc.ChromeOptions()
        prefs = {
            "download.default_directory": self.download_dir,
            "download.prompt_for_download": False,
            "download.directory_upgrade": True,
            "plugins.always_open_pdf_externally": True,
            "profile.default_content_settings.popups": 0,
        }
        options.add_experimental_option("prefs", prefs)
        options.add_argument("--no-first-run")
        options.add_argument("--password-store=basic")
        # options.add_argument("--headless=new") # 除錯時建議關閉 headless

        try:
            self.driver = uc.Chrome(options=options, use_subprocess=True)
            self.driver.set_window_size(1366, 768)
            
            # 強制修正下載路徑
            self.driver.execute_cdp_cmd("Page.setDownloadBehavior", {
                "behavior": "allow",
                "downloadPath": self.download_dir
            })
            
        except Exception as e:
            print(f"[錯誤] WebDriver 初始化失敗: {e}")
            raise
        self.wait = WebDriverWait(self.driver, 20)

    def _load_log(self) -> Tuple[Set[str], int]:
        urls, last_page = set(), 1
        try:
            with open(self.log_file, 'r', encoding='utf-8') as f:
                urls = {self._normalize_url(line.strip()) for line in f if line.strip() and self._normalize_url(line.strip())}
            print(f"[*] 已從 {self.log_file} 載入 {len(urls)} 筆有效紀錄。")
        except FileNotFoundError:
            print(f"[*] 未找到下載紀錄檔 {self.log_file}，將會從頭開始下載。")
        
        try:
            with open(self.page_progress_file, 'r', encoding='utf-8') as f:
                content = f.read().strip()
                if content.isdigit():
                    last_page = int(content)
                    print(f"[*] 已從 {self.page_progress_file} 載入上次爬取進度：從第 {last_page} 頁開始。")
                else:
                    print(f"[*] {self.page_progress_file} 內容無效，將從第 1 頁開始。")
        except FileNotFoundError:
            print(f"[*] 未找到頁數進度檔，將從第 1 頁開始。")
        return urls, last_page

    def _log_download(self, url: str):
        normalized_url = self._normalize_url(url)
        if not normalized_url: return
        with open(self.log_file, 'a', encoding='utf-8') as f: f.write(normalized_url + '\n')
        self.downloaded_urls.add(normalized_url)
        self.session_download_count += 1
        print(f"      - [計數] 本次執行已下載 {self.session_download_count}/{self.max_downloads_per_session} 篇。")

    def _log_progress(self, page_num: int):
        try:
            with open(self.page_progress_file, 'w', encoding='utf-8') as f: f.write(str(page_num))
        except Exception as e: print(f"[錯誤] 記錄頁數進度時發生錯誤: {e}")

    # =================================================================
    # ★★★ 終極版 Cookie 注射登入函式 (含 webnum) ★★★
    # =================================================================
    def wait_for_manual_login(self):
        print("\n[步驟 1] 啟動智慧登入程序 (三合一金鑰版)...")

        # 1. 嘗試 Cookie 注射
        print("[-] 正在執行 Cookie 身分移植...")
        try:
            self.driver.get(self.base_url)
            self.driver.delete_all_cookies()

            # ★ 完整的三把通關金鑰 ★
            cookies_to_inject = [
                {'name': 'ccd', 'value': '60n5Lm'},
                {'name': 'cookieaccount',
                    'value': 'ZG1DWWhJNlBjNGFnVTRPWWZvSnprbitqY2twN2VwNmFjM0tlaFhPV2RKcWNiV0pU'},
                {'name': 'webnum', 'value': '!qkHaA3Lx9NocAIB6myMi7ovTGDOvgb8OihAC1gbPp5uk8ST4m0/FVc6ByVK1IE4M82HQ9jHMfpEVTA=='}
            ]

            for cookie in cookies_to_inject:
                try:
                    self.driver.add_cookie(cookie)
                    print(f"    V 成功寫入: {cookie['name']}")
                except Exception as e:
                    print(f"    X 寫入失敗 {cookie['name']}: {e}")

            print("[-] 重新整理頁面，驗證身分...")
            self.driver.refresh()
            time.sleep(3)

            # 檢查是否成功登入
            if self.driver.find_elements(By.XPATH, "//div[@class='user_area']//a[text()='登出']"):
                print("\n[★] 完美！Cookie 注射成功！系統已認證您的身分。")
                print("    -> 自動跳過手動登入步驟，準備開始下載...")
                return  # 直接結束函式，開始下載
            else:
                print("[!] Cookie 狀態未更新 (可能需要 IP 配合)，轉為手動登入模式。")

        except Exception as e:
            print(f"[!] 注射過程發生錯誤 ({e})，轉為手動登入模式。")

        # -----------------------------------------------------------------
        # 2. 手動登入防線 (備用)
        # -----------------------------------------------------------------
        print("\n" + "="*60)
        print("★★★ 轉入與網站對抗模式 ★★★")
        print("1. 請手動輸入帳號密碼。")
        print("2. 程式會自動協助關閉錯誤警告並執行冷卻。")
        print("="*60 + "\n")

        end_time = time.time() + 900

        while time.time() < end_time:
            try:
                logout_btn = self.driver.find_elements(
                    By.XPATH, "//div[@class='user_area']//a[text()='登出']")
                if logout_btn:
                    print("\n[*] 偵測到登出按鈕，手動登入成功！")
                    return
                time.sleep(1)
            except (UnexpectedAlertPresentException, NoAlertPresentException):
                try:
                    alert = self.driver.switch_to.alert
                    msg = alert.text
                    print(f"\n[警告] 網站回傳訊息：{msg}")
                    critical_keywords = ["封鎖", "停用", "次數過多", "banned"]
                    if any(k in msg for k in critical_keywords):
                        alert.accept()
                        raise Exception(f"偵測到封鎖關鍵字: {msg}")
                    alert.accept()
                    print("      -> 強制冷卻 5 秒...")
                    for i in range(5, 0, -1):
                        print(f"         倒數 {i} 秒...", end="\r")
                        time.sleep(1)
                except:
                    pass
            except Exception:
                pass

        raise Exception("登入逾時。")

    def run_search(self):
        print("\n[步驟 2] 執行關鍵字搜尋 (修正欄位名稱版)...")

        # 1. 確保 Session ID 正確
        current_ccd = None
        match = re.search(r'ccd=([^/&]+)', self.driver.current_url)
        if match:
            current_ccd = match.group(1)
        else:
            try:
                cookie = self.driver.get_cookie('ccd')
                if cookie:
                    current_ccd = cookie['value']
            except:
                pass

        # 導航至正確頁面
        target_url = f"https://ndltd.ncl.edu.tw/cgi-bin/gs32/gsweb.cgi/ccd={current_ccd}/search?mode=basic"
        if current_ccd and (f"ccd={current_ccd}" not in self.driver.current_url or "search" not in self.driver.current_url):
            print(f"[-] 導航至搜尋頁面...")
            self.driver.get(target_url)

        try:
            # 等待搜尋框出現
            search_box = self.wait.until(
                EC.visibility_of_element_located((By.ID, "ysearchinput0")))

            # ★★★ 修正核心：將 ymeta 改為 dcf ★★★
            print("[-] 正在執行強制全選指令 (Target: dcf)...")
            self.driver.execute_script("""
                // 1. 抓取所有名稱為 'dcf' 的搜尋範圍框框
                var boxes = document.getElementsByName("dcf");
                
                // 2. 跑迴圈，全部強制勾選
                for (var i = 0; i < boxes.length; i++) {
                    if (!boxes[i].checked) {
                        boxes[i].checked = true;
                    }
                }
                
                // 3. (選用) 確保「電子全文」不被限制，這樣才能搜尋到紙本論文
                // 根據您的原始碼，電子全文的 ID 是 cacheinternet
                var fulltext = document.getElementById("cacheinternet");
                if (fulltext && fulltext.checked) {
                    fulltext.checked = false;
                }
            """)

            time.sleep(1)  # 讓您肉眼確認勾選狀態
            print("    V 全選指令執行完畢")

            # 輸入關鍵字 (含檢查)
            for i in range(3):
                search_box.clear()
                search_box.click()
                search_box.send_keys(self.keyword)
                time.sleep(0.5)
                if search_box.get_attribute('value') == self.keyword:
                    break
                print(f"    [!] 輸入重試第 {i+1} 次...")
                time.sleep(1)

            # 提交搜尋
            search_button = self.wait.until(
                EC.element_to_be_clickable((By.ID, "gs32search")))
            search_button.click()
            print(f"[*] 已成功提交搜尋，關鍵字為: '{self.keyword}'")

            # 解析結果
            try:
                print("[-] 正在等待總筆數資訊載入...")
                summary_container = self.wait.until(EC.visibility_of_element_located(
                    (By.XPATH, "//td[@headers='start' and contains(., '檢索結果共')]")))
                match = re.search(r'檢索結果共\s*(\d+)\s*筆資料',
                                  summary_container.text)
                if match:
                    total_items = int(match.group(1))
                    self.total_pages = math.ceil(
                        total_items / self.items_per_page)
                    print(
                        f"[*] 成功解析總筆數: {total_items} 筆，總頁數: {self.total_pages} 頁。")
                else:
                    raise NoSuchElementException
            except:
                print("[警告] 未能解析總筆數，將依賴「下一頁」按鈕判斷。")
                self.total_pages = 0

            # 跳頁邏輯 (維持不變)
            page_to_start = self.last_crawled_page
            if page_to_start > 1:
                if self.total_pages == 0 or page_to_start <= self.total_pages:
                    print(f"[*] 嘗試跳轉到第 {page_to_start} 頁...")
                    try:
                        jmpage_input = self.wait.until(
                            EC.visibility_of_element_located((By.ID, "jmpage")))
                        self.driver.execute_script(
                            "arguments[0].value = arguments[1];", jmpage_input, str(page_to_start))
                        jump_button = self.wait.until(
                            EC.element_to_be_clickable((By.NAME, "jumpfmt1page")))
                        old_html = self.driver.find_element(
                            By.TAG_NAME, 'html')
                        jump_button.click()
                        self.wait.until(EC.staleness_of(old_html))
                        print(f"[*] 成功跳轉到第 {page_to_start} 頁。")
                        time.sleep(3)
                    except Exception as e:
                        print(f"[錯誤] 跳頁失敗: {e}，從頭開始。")
                        self.last_crawled_page = 1
                else:
                    self.last_crawled_page = 1
        except TimeoutException:
            print("[錯誤] 搜尋頁面逾時。")
            raise


    def _sanitize_filename(self, name: str) -> str:
        sanitized_name = re.sub(r'[\\/*?:"<>|]', "", name)
        return sanitized_name.strip()[:150]

    def _parse_article_links(self) -> List[Tuple[str, str]]:
        results = []
        try:
            self.wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "td.tdfmt1-content")))
            article_elements = self.driver.find_elements(By.CSS_SELECTOR, "td.tdfmt1-content")
            for elem in article_elements:
                try:
                    link_tag = elem.find_element(By.CSS_SELECTOR, "a.slink")
                    title_span = link_tag.find_element(By.CSS_SELECTOR, "span.etd_d")
                    url = link_tag.get_attribute('href')
                    title = title_span.text
                    if url and title: results.append((url, title))
                except NoSuchElementException: continue
        except TimeoutException: print("[警告] 等待論文連結載入逾時。")
        return results

    def _wait_for_download_complete(self, timeout: int = 180) -> Optional[str]:
        print("      - 自動監控下載中...", end="")
        seconds, initial_dl_files = 0, set(os.listdir(self.download_dir))
        while seconds < timeout:
            new_files = set(os.listdir(self.download_dir)) - initial_dl_files
            if new_files:
                candidates = [f for f in new_files if not f.endswith('.crdownload')]
                if candidates:
                    new_file_name = candidates[0]
                    full_path = os.path.join(self.download_dir, new_file_name)
                    time.sleep(1) 
                    print(f" 下載完成: {new_file_name}")
                    return full_path
            time.sleep(1)
            seconds += 1
            if seconds % 10 == 0: print(".", end="", flush=True)
        print("\n      - [錯誤] 等待下載逾時。")
        return None

    def _preprocess_captcha_image(self, image_bytes: bytes) -> bytes:
        try:
            img = Image.open(BytesIO(image_bytes)).convert('L').point(lambda p: 255 if p > 128 else 0)
            buffered = BytesIO()
            img.save(buffered, format="PNG")
            return buffered.getvalue()
        except: return image_bytes

    def _solve_captcha_with_ddddocr(self, captcha_element) -> str:
        try:
            res = self.ocr.classification(self._preprocess_captcha_image(captcha_element.screenshot_as_png))
            res_cleaned = ''.join(filter(str.isalnum, res)).lower()
            print(f"      - ddddocr: '{res_cleaned}'")
            if 4 <= len(res_cleaned) <= 6: return res_cleaned
            return ""
        except: return ""

    def _unzip_and_cleanup(self, file_path: str, new_name_base: str):
        if not file_path.lower().endswith('.zip'): return
        new_pdf_name = f"{new_name_base}.pdf"
        dest_pdf_path = os.path.join(self.download_dir, new_pdf_name)
        print(f"      - 解壓縮並改名為: {new_pdf_name}")
        try:
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                pdfs = [n for n in zip_ref.namelist() if n.lower().endswith('.pdf')]
                if not pdfs: return
                with zip_ref.open(pdfs[0]) as source, open(dest_pdf_path, 'wb') as target:
                    shutil.copyfileobj(source, target)
            os.remove(file_path)
        except Exception as e: print(f"      - [錯誤] 解壓失敗: {e}")

    def _handle_alert_if_present(self) -> bool:
        try:
            alert = self.driver.switch_to.alert
            alert.accept()
            print("      - 警告視窗已關閉。")
            return True
        except NoAlertPresentException: return False
        except UnexpectedAlertPresentException: # 額外防護
            try:
                self.driver.switch_to.alert.accept()
                return True
            except: return False

    def _process_article_in_new_tab(self, article_url: str, article_title: str):
        print(f"    - 正在處理: {article_title}")
        self.driver.switch_to.new_window('tab')
        self.driver.get(article_url)
        MAX_RETRIES = 3
        try:
            self.wait.until(EC.element_to_be_clickable((By.XPATH, "//a[em[text()='電子全文']]"))).click()
            self.wait.until(EC.element_to_be_clickable((By.XPATH, "//img[@alt='電子全文']/following-sibling::a[@title='電子全文']"))).click()
            time.sleep(random.uniform(1.5, 3.0))
            self.driver.switch_to.window(self.driver.window_handles[-1])
            
            for i in range(MAX_RETRIES):
                print(f"      - 嘗試下載 (第 {i + 1}/{MAX_RETRIES} 次)...")
                try:
                    captcha_img = self.wait.until(EC.presence_of_element_located((By.XPATH, "//img[contains(@src, 'random_validation')]")))
                    captcha_text = self._solve_captcha_with_ddddocr(captcha_img)
                    if not captcha_text:
                        self.driver.refresh(); time.sleep(2); continue
                    
                    input_box = self.driver.find_element(By.ID, "validinput")
                    input_box.clear()
                    input_box.send_keys(captcha_text)
                    time.sleep(1)
                    
                    self.driver.find_element(By.XPATH, "//input[@value='我同意']").click()
                    time.sleep(1.5)
                    
                    if self._handle_alert_if_present():
                        self.driver.refresh(); time.sleep(2); continue
                    
                    self.wait.until(EC.presence_of_element_located((By.LINK_TEXT, "下載"))).click()
                    
                    newly_downloaded_file = self._wait_for_download_complete()
                    if newly_downloaded_file:
                        self._log_download(article_url)
                        sanitized_title = self._sanitize_filename(article_title)
                        if newly_downloaded_file.lower().endswith(".zip"):
                            self._unzip_and_cleanup(newly_downloaded_file, sanitized_title)
                        elif newly_downloaded_file.lower().endswith(".pdf"):
                            new_pdf_path = os.path.join(self.download_dir, f"{sanitized_title}.pdf")
                            print(f"      - 改名為: {sanitized_title}.pdf")
                            if os.path.exists(new_pdf_path):
                                base, ext = os.path.splitext(new_pdf_path)
                                new_pdf_path = f"{base}_{int(time.time())}{ext}"
                            os.rename(newly_downloaded_file, new_pdf_path)
                        return
                    else:
                        print("      - [警告] 下載逾時。")
                        break
                except Exception as e:
                    print(f"      - [重試 {i+1} 錯誤] {e}")
                    if self._handle_alert_if_present(): pass
                    if i < MAX_RETRIES - 1:
                        self.driver.refresh(); time.sleep(3)
                    else: print("      - 放棄此論文。")
        except TimeoutException:
            print("      - [提示] 無法找到電子全文按鈕或下載失敗。")
        except Exception as e:
            print(f"      - [錯誤] {e}")
        finally:
            while len(self.driver.window_handles) > 1:
                self.driver.switch_to.window(self.driver.window_handles[-1])
                self.driver.close()
            self.driver.switch_to.window(self.main_window_handle)
            time.sleep(random.uniform(*self.inter_article_sleep_range))

    def run_download_process(self):
        print("\n[步驟 3] 執行下載流程...")
        if not self.main_window_handle: self.main_window_handle = self.driver.current_window_handle
        page_num = self.last_crawled_page
        while True:
            if self.session_download_count >= self.max_downloads_per_session:
                print("已達下載上限。")
                self._log_progress(page_num)
                break
            
            print(f"\n--- 第 {page_num} 頁 ---")
            self.driver.switch_to.window(self.main_window_handle)
            try: self.wait.until(EC.presence_of_element_located((By.ID, "tablefmt1")))
            except TimeoutException: break
            
            article_urls = self._parse_article_links()
            print(f"[*] 本頁找到 {len(article_urls)} 篇。")
            
            for url, title in article_urls:
                if self.session_download_count >= self.max_downloads_per_session: break
                normalized = self._normalize_url(url)
                if not normalized: continue
                if normalized in self.downloaded_urls:
                    print(f"    - [跳過] 已下載: {title}")
                    continue
                self._process_article_in_new_tab(url, title)
            
            self._log_progress(page_num)
            try:
                next_btn = self.wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input[name="gonext"][type="image"]:not([src*="_"])')))
                self.driver.execute_script("arguments[0].click();", next_btn)
                page_num += 1
                time.sleep(random.uniform(*self.inter_page_sleep_range))
            except:
                print("沒有下一頁了。")
                break

    def run(self):
        try:
            self._setup_driver()
            self.wait_for_manual_login()
            self.run_search()
            self.run_download_process()
        finally:
            self.close()

    def close(self):
        if self.driver:
            self.driver.quit()
            self.driver = None

class ThesisDownloaderWithReadme(BaseThesisDownloader):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.readme_file = os.path.join(BASE_DIR, "README.md")
        self.readme_handle = None

    def _initialize_readme(self):
        try:
            self.readme_handle = open(self.readme_file, 'a+', encoding='utf-8')
            self.readme_handle.seek(0)
            if not self.readme_handle.read(1):
                self.readme_handle.write(f"# 下載清單 - {self.keyword}\n\n")
        except: pass

    def _consolidate_existing_pdfs(self):
        print("\n[統整模式] 檢查已存在的 PDF...")
        if not self.readme_handle: return
        try:
            existing = {f for f in os.listdir(self.download_dir) if f.endswith('.pdf')}
            self.readme_handle.seek(0)
            content = self.readme_handle.read()
            for f in existing:
                if f not in content:
                    self.readme_handle.write(f"* {f} (已存在)\n")
            self.readme_handle.flush()
        except: pass

    def _setup_driver(self):
        print("[-] 設定 Selenium WebDriver (記憶模式 - Persistent Profile)...")

        # 1. 設定下載路徑
        os.makedirs(self.download_dir, exist_ok=True)

        # 2. 設定「瀏覽器記憶設定檔」的路徑
        # 這樣做會讓 Selenium 讀寫這個資料夾，像真人的瀏覽器一樣記住 Cookies 和登入狀態
        profile_path = os.path.join(BASE_DIR, "chrome_profile_thesis")
        if not os.path.exists(profile_path):
            os.makedirs(profile_path)
            print(f"[*] 首次建立瀏覽器設定檔資料夾: {profile_path}")
        else:
            print(f"[*] 讀取現有的瀏覽器設定檔: {profile_path}")

        # 3. 設定 Chrome 選項
        options = uc.ChromeOptions()

        # ★★★ 關鍵修改：指定 User Data Dir (讓瀏覽器長記性) ★★★
        options.add_argument(f'--user-data-dir={profile_path}')

        # 其他優化設定
        prefs = {
            "download.default_directory": self.download_dir,
            "download.prompt_for_download": False,
            "download.directory_upgrade": True,
            "plugins.always_open_pdf_externally": True,
            "profile.default_content_settings.popups": 0,
            # 嘗試關閉自動翻譯與其他干擾
            "translate_whitelists": {"zh-TW": "en"},
            "translate.enabled": False
        }
        options.add_experimental_option("prefs", prefs)

        options.add_argument("--no-first-run")
        options.add_argument("--password-store=basic")
        # options.add_argument("--headless=new") # 登入階段絕對不要開 headless

        try:
            # 啟動瀏覽器
            self.driver = uc.Chrome(options=options, use_subprocess=True)
            self.driver.set_window_size(1366, 768)

            # 強制修正下載路徑 (雙重保險)
            self.driver.execute_cdp_cmd("Page.setDownloadBehavior", {
                "behavior": "allow",
                "downloadPath": self.download_dir
            })

        except Exception as e:
            print(f"[錯誤] WebDriver 初始化失敗: {e}")
            print(
                "[提示] 若出現 'user data directory is already in use' 錯誤，請先關閉所有 Chrome視窗 或 重啟核心 (Kernel)。")
            raise

        self.wait = WebDriverWait(self.driver, 20)

    def run(self):
        try:
            self._setup_driver()
            self._consolidate_existing_pdfs()
            self.wait_for_manual_login()
            self.run_search()
            self.run_download_process()
        finally:
            self.close()
            if self.readme_handle: self.readme_handle.close()

if __name__ == "__main__":
    # ★ 建議：如果之前被封鎖，請先重啟路由器換 IP 後再執行
    downloader = ThesisDownloaderWithReadme(
        keyword="精神疾病 家庭動力",
        max_downloads_per_session=200
    )
    downloader.run()

[*] 已從 c:\Users\user\Documents\GitHub\taiwan_essay_download\download_log.txt 載入 429 筆有效紀錄。
[*] 已從 c:\Users\user\Documents\GitHub\taiwan_essay_download\page_progress.txt 載入上次爬取進度：從第 4 頁開始。
[-] 正在初始化 ddddocr 引擎...
[*] ddddocr 引擎初始化完成。
[*] 本次執行最大下載量設定為: 200 篇
[-] 設定 Selenium WebDriver (記憶模式 - Persistent Profile)...
[*] 讀取現有的瀏覽器設定檔: c:\Users\user\Documents\GitHub\taiwan_essay_download\chrome_profile_thesis

[統整模式] 檢查已存在的 PDF...

[步驟 1] 啟動智慧登入程序 (三合一金鑰版)...
[-] 正在執行 Cookie 身分移植...
    V 成功寫入: ccd
    V 成功寫入: cookieaccount
    V 成功寫入: webnum
[-] 重新整理頁面，驗證身分...
[!] Cookie 狀態未更新 (可能需要 IP 配合)，轉為手動登入模式。

★★★ 轉入與網站對抗模式 ★★★
1. 請手動輸入帳號密碼。
2. 程式會自動協助關閉錯誤警告並執行冷卻。



KeyboardInterrupt: 



精神疾病 + 家庭動力

Bowen + 自我分化 + 台灣

結構派 + 家庭治療 + 界限


心理衛生社工 + 社會安全網

多重問題家庭 + 處遇 (或 多重需求家庭)

高風險家庭 + 系統觀點

社區精神復健 + 家庭工作


家族治療 + 本土化

華人家庭 + 溝通模式

家庭界限 + 文化

精神障礙者 + 照顧負荷 + 家庭韌性


